# Making Pandas DataFrames from API Requests
In this example, we will use the U.S. Geological Survey's API to grab a JSON object of earthquake data and convert it to a pandas DataFrame.

USGS API: https://earthquake.usgs.gov/fdsnws/event/1/

### Get Data from API

In [1]:
import datetime
import pandas as pd
import requests

yesterday = datetime.date.today() - datetime.timedelta(days=1)
api = 'https://earthquake.usgs.gov/fdsnws/event/1/query'
payload = {
    'format' : 'geojson',
    'starttime' : yesterday - datetime.timedelta(days=26),
    'endtime' : yesterday
}
response = requests.get(api, params=payload)

# let's make sure the request was OK
print(response.status_code)

200


Response of 200 means OK, so we can pull the data out of the result. Since we asked the API for a JSON, we can extract it from the response with the JSON method.

### Isolate the Data from the JSON Response
We need to check the structures of the response data to know where our data is.

In [2]:
earthquake_json = response.json()
earthquake_json.keys()

dict_keys(['type', 'metadata', 'features', 'bbox'])

In [3]:
earthquake_json['metadata']

{'generated': 1751722769000,
 'url': 'https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2025-06-08&endtime=2025-07-04',
 'title': 'USGS Earthquakes',
 'status': 200,
 'api': '1.14.1',
 'count': 8466}

Each element in the JSON array `features` is a row of data for our dataframe.

In [4]:
type(earthquake_json['features'])

list

Your data will be different depending on the date you run this.

In [5]:
earthquake_json['features'][0]

{'type': 'Feature',
 'properties': {'mag': 0.71,
  'place': '8 km WNW of Cobb, CA',
  'time': 1751587044680,
  'updated': 1751587142559,
  'tz': None,
  'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/nc75204211',
  'detail': 'https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=nc75204211&format=geojson',
  'felt': None,
  'cdi': None,
  'mmi': None,
  'alert': None,
  'status': 'automatic',
  'tsunami': 0,
  'sig': 8,
  'net': 'nc',
  'code': '75204211',
  'ids': ',nc75204211,',
  'sources': ',nc,',
  'types': ',nearby-cities,origin,phase-data,',
  'nst': 14,
  'dmin': 0.01378,
  'rms': 0.01,
  'gap': 59,
  'magType': 'md',
  'type': 'earthquake',
  'title': 'M 0.7 - 8 km WNW of Cobb, CA'},
 'geometry': {'type': 'Point',
  'coordinates': [-122.811164855957, 38.8371658325195, 1.85000002384186]},
 'id': 'nc75204211'}

### Convert to DataFrame
We need to grab the `properties` section out of every entry in the `features` JSON array to create our dataframe.

In [6]:
earthquake_properties_data = [data['properties'] for data in earthquake_json['features']]
df = pd.DataFrame(earthquake_properties_data)
df.head()

,alert,cdi,code,detail,dmin,felt,gap,ids,mag,magType,...,sources,status,time,title,tsunami,type,types,tz,updated,url
0,None,NaN,75204211,https://earthquake.usgs.gov/fdsnws/event/1/que...,0.01378,NaN,59.0,",nc75204211,",0.71,md,...,",nc,",automatic,1751587044680,"M 0.7 - 8 km WNW of Cobb, CA",0,earthquake,",nearby-cities,origin,phase-data,",None,1751587142559,https://earthquake.usgs.gov/earthquakes/eventp...
1,None,NaN,2025mztjkv,https://earthquake.usgs.gov/fdsnws/event/1/que...,0.00000,NaN,106.0,",tx2025mztjkv,",1.20,ml,...,",tx,",automatic,1751586762405,"M 1.2 - 53 km SSW of Whites City, New Mexico",0,earthquake,",origin,phase-data,",None,1751587263766,https://earthquake.usgs.gov/earthquakes/eventp...
2,None,NaN,75204206,https://earthquake.usgs.gov/fdsnws/event/1/que...,0.01051,NaN,60.0,",nc75204206,",1.02,md,...,",nc,",automatic,1751586557620,"M 1.0 - 7 km WNW of Cobb, CA",0,earthquake,",nearby-cities,origin,phase-data,",None,1751586654103,https://earthquake.usgs.gov/earthquakes/eventp...
3,None,NaN,2025mztemw,https://earthquake.usgs.gov/fdsnws/event/1/que...,0.10000,NaN,64.0,",tx2025mztemw,",1.20,ml,...,",tx,",reviewed,1751586403566,"M 1.2 - 29 km SW of Garden City, Texas",0,earthquake,",origin,phase-data,",None,1751623733404,https://earthquake.usgs.gov/earthquakes/eventp...
4,None,NaN,41013895,https://earthquake.usgs.gov/fdsnws/event/1/que...,0.01720,NaN,26.0,",ci41013895,",1.09,ml,...,",ci,",automatic,1751586190420,"M 1.1 - 4 km NW of San Jacinto, CA",0,earthquake,",nearby-cities,origin,phase-data,scitech-link,",None,1751586401167,https://earthquake.usgs.gov/earthquakes/eventp...


### (Optional) Write Data to CSV

In [7]:
pd.DataFrame(earthquake_properties_data).to_csv('earthquakes.csv', index=False)